{{ badge }}

You can refer the original post here:

https://machinelearningmastery.com/handle-missing-data-python/

**<h1>How to handle Missing Data with python**

**<h3>Introduction**

* First of all why values will be missing in the dataset given?
* This can happen for various number of reasons such as observations not recorded at all may be due to cost issue for collecting, data got corrupted, data engineers didn't scrape the data well from sources.
* Handling missing values is such important as many ML algorithms do not tolerate data with missing values.

This post discusses below things:
  * Observing dataset
  * Mark missing values
  * Problems caused by missing values
  * Removing rows with missing values
  * Imputing missing values
  * Looking at algos that support missing values

**<h3>1. Diabetes Dataset**

In [ ]:
import getpass as gt

user = input('Provide Kaggle Username : ')
api = gt.getpass('API key for '+ user +' :')
print('')


!pip install kaggle
!kaggle -v

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

kaggle_api_key = {'username':user, 'key':api}

import json

with open('/root/.kaggle/kaggle.json','w') as file:
  json.dump(kaggle_api_key,file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d uciml/pima-indians-diabetes-database


import pandas as pd
from zipfile import ZipFile

diabetes_zip = ZipFile('/content/pima-indians-diabetes-database.zip')
train_df = pd.read_csv(diabetes_zip.open('diabetes.csv'))

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
mkdir: cannot create directory ‘/root/.kaggle’: File exists
  0% 0.00/8.91k [00:00<?, ?B/s]
100% 8.91k/8.91k [00:00<00:00, 17.7MB/s]


In [ ]:
train_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
train_df.shape

(768, 9)

In [ ]:
train_df['Outcome'].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [ ]:
maj_cls = train_df['Outcome'][train_df['Outcome']==0].count()
min_cls = train_df['Outcome'][train_df['Outcome']==1].count()
tot_pts = train_df.shape[0]

print('Size of dataset ' , tot_pts)
print('Majority class pionts ', maj_cls)
print('Minority class pionts ', min_cls)
print('Distribution of majority class (0) over data %.1f%%'%(maj_cls/tot_pts*100))
print('Distribution of minority class (1) over data %.1f%%'%(min_cls/tot_pts*100))

Size of dataset  768
Majority class pionts  500
Minority class pionts  268
Distribution of majority class (0) over data 65.1%
Distribution of minority class (1) over data 34.9%


* This is a binary classification dataset and distribution of data for each class is not balanced.
* Size of data is 768, 8 features and 1 output variable.
* If we can see the head data has value for some columns as '0' which indicates these are missing values.

**<h3>2. Mark missing values**

* As the data size increases, no. of missing values also increases.
* We can use plots and summary statistics to identify missing or corrupt data.

In [ ]:
train_df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


* From above description we can see that minimum of some columns is '0' which is missing or corrupted data.
* We are going to confirm this by looking at raw data.

In [ ]:
num_missing = (train_df[['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI']]==0).sum()

print(num_missing)

Pregnancies      111
Glucose            5
BloodPressure     35
SkinThickness    227
Insulin          374
BMI               11
dtype: int64


* We can see that columns 'Glucose','BloodPressure','BMI' have few points as zero whereas columns 'Insulin' has half of the rows as zeros and other columns as well.
* We can't just simply delete those rows as we'll loose most of the data, so we need some strategies to fill those columns with reasonable values.
* As we are seeing missing values as '0', we mark them with Nan(null) to identify easily.
* This marking helps us to replace them easily with reasonable values using some python functions.

In [ ]:
from numpy import nan

train_df[['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = train_df[['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,nan)

#counting no. of nan values
print(train_df.isnull().sum())


Pregnancies                 111
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64


* Lets confirm this Nan values by looking at raw data

In [ ]:
train_df.head(20)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.0,148.0,72.0,35.0,NaN,33.6,0.627,50,1
1,1.0,85.0,66.0,29.0,NaN,26.6,0.351,31,0
2,8.0,183.0,64.0,NaN,NaN,23.3,0.672,32,1
3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,NaN,137.0,40.0,35.0,168.0,43.1,2.288,33,1
5,5.0,116.0,74.0,NaN,NaN,25.6,0.201,30,0
6,3.0,78.0,50.0,32.0,88.0,31.0,0.248,26,1
7,10.0,115.0,NaN,NaN,NaN,35.3,0.134,29,0
8,2.0,197.0,70.0,45.0,543.0,30.5,0.158,53,1
9,8.0,125.0,96.0,NaN,NaN,NaN,0.232,54,1


**<h3>3. Missing Values Causes Problems**

* Due to this existing missing values in dataset cause some Machine learning algorithms go insane.
* Most missing values are common in real world data and unfortunately many ml algorithms can't handle these. Therefore, this problem needs to be addressed prior to modelling.
* To check the errors, we will try Linear Discriminant Analysis (LDA) algorithm on the dataset with missing values.
* We tried this particular algorithm, because LDA doesn't work with data having missing values.
* We already marked the missing values before, we use the same dataset for analysis.
* We evaluate LDA using 3-fold cv and print mean ***accuracy***.

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

#splitting dataset into inputs and outputs
values = train_df.values
X = values[:,0:8]
y = values[:,8]

#defining model
model_LDA = LinearDiscriminantAnalysis()

cv = KFold(n_splits=3, shuffle=True, random_state=1)

#evaluating the model
result = cross_val_score(model_LDA, X,y, cv=cv, scoring = 'accuracy')

#report the mean performance
print('Accuracy: %.3f' %result.mean())

Accuracy: nan


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

  FitFailedWarning)


* It is clear from above error that missing values creates errors in some models.

**<h3>4. Remove rows with Missing Values**

* Most simplest way for handling missing data.
* We remove entire row that if a column of it contains missing value using ***dropna()***.

In [ ]:
from numpy import nan

train_df[['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = train_df[['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,nan)

#dropping rows with missing values
train_df_rm = train_df.dropna()
print('Shape before dropping : ', train_df.shape)
print('Shape after dropping : ',train_df_rm.shape)

Shape before dropping :  (768, 9)
Shape after dropping :  (336, 9)


* Now we use the modified dataset to test LDA algorithm.

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

#splitting dataset into inputs and outputs
values = train_df_rm.values
X = values[:,0:8]
y = values[:,8]

#defining model
model_LDA = LinearDiscriminantAnalysis()

cv = KFold(n_splits=3, shuffle=True, random_state=1)

#evaluating the model
result = cross_val_score(model_LDA, X,y, cv=cv, scoring = 'accuracy')

#report the mean performance
print('Accuracy: %.3f' %result.mean())

Accuracy: 0.777


* The example run successfully and accuracy got returned.
* But removing the rows is not a good idea all the time as in the current case we lost 50% of data.
* So, following we are going to look at some imputation techniques.

**<h3>5. Impute Missing Values**

* Imputing referes to using a model to replace the missing values.
* We can use training set predictors to estimate the values of other predictors.
* Options we have for imputation:
  * A constant value that has meaning within the domain, eg: 0 (distinct from all other values)
  * A value from another randomly selected record.
  * A mean, median or mode value for the column.
  * A value estimated by another predictive model.
* Any method we choose to perfom on training model will have to be performed on unseen test data as well if test data has also missing values.
* Eg: If we choose to impute mean column values, these column values will need to be stored to file for later use on new data that has missin values.
* We use ***fillna()*** method to impute missin values.

In [ ]:
from numpy import nan

#load dataset
train_df = pd.read_csv(diabetes_zip.open('diabetes.csv'))

#marking missing values
train_df[['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = train_df[['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,nan)

#imputing missing values with mean column values
train_df.fillna(train_df.mean(), inplace=True)

#count number of missing values in each column
print(train_df.isnull().sum())

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


* The above procedure has a risk of data leakage where we did imputation before splitting the data for testing and cross validation.
* So we use ***Pipeline*** where data is first passed through the imputer transform such as ***SimpleImputer*** then provided to the model.
* This way we can be sure that imputer and model are both fit only on the training dataset and evaluated on test dataset within each cross-validation fold thus avoiding data leakage.

In [ ]:
from numpy import nan
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

#load dataset
train_df = pd.read_csv(diabetes_zip.open('diabetes.csv'))

#marking missing values
train_df[['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = train_df[['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,nan)


#splitting dataset into inputs and outputs
values = train_df_rm.values
X = values[:,0:8]
y = values[:,8]

#define the imputer
imputer = SimpleImputer(mising_values=nan, strategy='mean')

#defining model
model_LDA = LinearDiscriminantAnalysis()

#defining the model pipeline
pipeline = Pipeline(steps=[('imputer', imputer), ('model', model_LDA)])

kfold = KFold(n_splits=3, shuffle=True, random_state=1)

#evaluating the model
result = cross_val_score(pipeline, X,y, cv=kfold, scoring = 'accuracy')

#report the mean performance
print('Accuracy: %.3f' %result.mean())


Pregnancies                   4.494673
Glucose                     121.686763
BloodPressure                72.405184
SkinThickness                29.153420
Insulin                     155.548223
BMI                          32.457464
DiabetesPedigreeFunction      0.471876
Age                          33.240885
Outcome                       0.348958
dtype: float64

**<h3>6. Algorithms that Support Missing Values**

* Not all algorithms fail when there's missing data.
* Eg: KNN ignore a column from a distance measure when a value is missing.<br>NB also support missing values when makind a prediction.
* There are also algorithms that use missinng values as a unique and different value when building the predictive model such as *classification* and *regression trees*.
* But scikit-learn implementation of naive bayes, DT and KNN are not robust to missing values.

**<h3>7. Imputing missing values using model predictions**

* This is one of the best and most efficient method for handling missing data.
* Depending on the class of data that is missing, we either use classification or regression model to predict missing data.
* We turn features with missing values as 'Output', making rest of the features (with all the columns data available) as predicting columns.
* Call the feature that has missing values as *y*.
* Split the data into sets with missing values and without missing values. Name the missing set 'X_test' and without missing 'X_train' and take *y* off the second set, naming it as y_train.
* After predicting missing values, add it to X_test as your y_test column, then combine sets together.
* The only drawback to this approach is that if there is no correlation between features with missing data and other features which we use for prediction, then the model will be bias for predicting missing values.